
# Zane Bates
# CMU 42640 — Project
**Course:** Image-Based Computational Modeling and Analysis  

**Problems:** 1) MRI dog brain volume from `.rawiv` subunits; 2) Electrostatic potential on protein isosurface via **trilinear interpolation**.


## Problem 1 — Volume of Segmented Dog Brain

In [ ]:
import struct
import numpy as np

def read_rawiv(path):
    # Read header (68 bytes, big-endian)
    with open(path, 'rb') as f:
        header = f.read(68)
        minX, minY, minZ, maxX, maxY, maxZ, numVerts, numCells, dimX, dimY, dimZ, originX, originY, originZ, spanX, spanY, spanZ = struct.unpack('>6f2I3I3f3f', header)
        
        # Read data
        f.seek(68)
        data = np.fromfile(f, dtype='>u1', count=numVerts)
    
    # Reshape to 3D volume
    volume = data.reshape((dimZ, dimY, dimX))
    
    return volume, spanX, spanY, spanZ

def calculate_volume(path, threshold=0):
    volume, spanX, spanY, spanZ = read_rawiv(path)
    
    # Count voxels above threshold
    num_voxels = np.sum(volume > threshold)
    
    # Calculate volume in mm^3
    voxel_volume = spanX * spanY * spanZ
    total_volume = num_voxels * voxel_volume
    
    return total_volume, num_voxels

# file paths
path1 = r"D:\repos\cmu-hw\image_based_comp\project_1\Project1\mri_dcm.rawiv_subunit_01.rawiv"
path2 = r"D:\repos\cmu-hw\image_based_comp\project_1\Project1\mri_dcm.rawiv_subunit_02.rawiv"

# Calculate volumes
vol1, n1 = calculate_volume(path1)
vol2, n2 = calculate_volume(path2)

total = vol1 + vol2

print(f"Subunit 1 (Total Brain volume): {n1:,} voxels, {vol1:,.2f} mm³ ({vol1/1000:.3f} mL)")
print(f"Subunit 2 (Total Outside Brain Volume): {n2:,} voxels, {vol2:,.2f} mm³ ({vol2/1000:.3f} mL)")
print(f"\nTotal Volume: {total:,.2f} mm³ ({total/1000:.3f} mL)")

Subunit 1 (Total Brain volume): 61,943 voxels, 91,496.68 mm³ (91.497 mL)
Subunit 2 (Total Outside Brain Volume): 835,110 voxels, 1,233,550.02 mm³ (1233.550 mL)

Total Volume: 1,325,046.70 mm³ (1325.047 mL)



---
## Problem 2 — Electrostatic Potential on Isosurface


In [ ]:
import struct
import numpy as np

# Read RAWIV
def read_rawiv(path):
    with open(path, 'rb') as f:
        header = f.read(68)
        minX, minY, minZ, maxX, maxY, maxZ, numVerts, numCells, dimX, dimY, dimZ, originX, originY, originZ, spanX, spanY, spanZ = struct.unpack('>6f2I3I3f3f', header)
        
        f.seek(68)
        data = np.fromfile(f, dtype='>f4', count=dimX*dimY*dimZ)
        vol = data.reshape((dimZ, dimY, dimX))
    
    return vol, (minX, minY, minZ), (spanX, spanY, spanZ), (dimX, dimY, dimZ)

# Load triangle mesh from .raw file
def load_raw_mesh(path):
    with open(path, 'r') as f:
        nVerts, nTris = map(int, f.readline().split())
        vertices = np.loadtxt(f, max_rows=nVerts)
        triangles = np.loadtxt(f, dtype=int)
    
    return vertices, triangles

# Interpolate potentials
def interpolate(vol, mins, spans, dims, xyz):
    minX, minY, minZ = mins
    spanX, spanY, spanZ = spans
    dimX, dimY, dimZ = dims
    
    fx = (xyz[:,0] - minX) / spanX
    fy = (xyz[:,1] - minY) / spanY
    fz = (xyz[:,2] - minZ) / spanZ
    
    ix = np.clip(np.floor(fx).astype(int), 0, dimX-2)
    iy = np.clip(np.floor(fy).astype(int), 0, dimY-2)
    iz = np.clip(np.floor(fz).astype(int), 0, dimZ-2)
    
    u, v, w = fx - ix, fy - iy, fz - iz
    
    f000 = vol[iz,   iy,   ix  ]; f100 = vol[iz,   iy,   ix+1]
    f010 = vol[iz,   iy+1, ix  ]; f110 = vol[iz,   iy+1, ix+1]
    f001 = vol[iz+1, iy,   ix  ]; f101 = vol[iz+1, iy,   ix+1]
    f011 = vol[iz+1, iy+1, ix  ]; f111 = vol[iz+1, iy+1, ix+1]
    
    return (f000*(1-u)*(1-v)*(1-w) + f100*u*(1-v)*(1-w) +
            f010*(1-u)*v*(1-w) + f110*u*v*(1-w) +
            f001*(1-u)*(1-v)*w + f101*u*(1-v)*w +
            f011*(1-u)*v*w + f111*u*v*w)

# Map potential to RGB color (blue -> red)
def potential_to_color(potentials):
    pmin, pmax = potentials.min(), potentials.max()
    normalized = (potentials - pmin) / (pmax - pmin)
    
    colors = np.zeros((len(potentials), 3))
    colors[:, 0] = normalized      # Red channel
    colors[:, 2] = 1 - normalized  # Blue channel
    
    return colors

# File paths
pot_file = r"D:\repos\cmu-hw\image_based_comp\project_1\Project1\2BG9_pot97129.rawiv"
mesh_file = r"D:\repos\cmu-hw\image_based_comp\project_1\Project1\mol_surf.raw"
output_file = mesh_file.replace('.raw', '_tri.rawc')

# Load and process
vol, mins, spans, dims = read_rawiv(pot_file)
vertices, triangles = load_raw_mesh(mesh_file)
potentials = interpolate(vol, mins, spans, dims, vertices)
colors = potential_to_color(potentials)

print(f"Potential min/mean/max: {potentials.min():.6f}, {potentials.mean():.6f}, {potentials.max():.6f}")

# write .rawc file
with open(output_file, 'w') as f:
    f.write(f"{len(vertices)} {len(triangles)}\n")
    
    for v, c in zip(vertices, colors):
        f.write(f"{v[0]} {v[1]} {v[2]} {c[0]} {c[1]} {c[2]}\n")
    
    for tri in triangles:
        f.write(f"{tri[0]} {tri[1]} {tri[2]}\n")

print(f"Saved: {output_file}")

Potential min/mean/max: -688.536785, 5.259720, 485.026063
Saved: D:\repos\cmu-hw\image_based_comp\project_1\Project1\mol_surf_tri.rawc
